In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from openpyxl import Workbook
import datetime
import time
import os


In [2]:
# WebDriver 설정
options = Options()
options.add_argument("window-size=1920x1080")
options.add_argument("--disable-blink-features=AutomationControlled")
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

In [3]:
# 네이버플레이스 모바일 리뷰 URL
driver.get("https://m.place.naver.com/restaurant/98516235/review/visitor?reviewSort=recent")
driver.implicitly_wait(10)


In [8]:
# 엑셀 설정
now = datetime.datetime.now()
wb = Workbook()
if 'Sheet' in wb.sheetnames:
    del wb['Sheet']
sheet = wb.create_sheet('output')
sheet.append(['nickname', 'content', 'date', 'revisit', 'tags', 'review_cnt', 'url'])


In [10]:
from selenium.webdriver.common.keys import Keys

try:
    for _ in range(30):  # 최대 30번 반복
        try:
            driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.END)
            time.sleep(1.5)

            more_button = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, 'a.fvwqf'))
            )
            driver.execute_script("arguments[0].click();", more_button)
            time.sleep(2)
        except Exception as e:
            print("📌 더보기 버튼 없음 or 클릭 실패:", e)
            break
except Exception as e:
    print("❌ 반복 중 오류:", e)

In [11]:
    # 모든 페이지가 로드되었으므로, 이제 리뷰 데이터와 태그를 수집
    reviews = driver.find_elements(By.CSS_SELECTOR, 'li.pui__X35jYm.EjjAW')

In [49]:
for r in reviews:
        nickname = r.find_element(By.CSS_SELECTOR, 'span.pui__uslU0d').text.strip()
        content = r.find_element(By.CSS_SELECTOR, 'div.pui__vn15t2').text.strip()
        date = r.find_element(By.CSS_SELECTOR, 'span.pui__gfuUIT > time').text.strip()
        revisit_elements = r.find_elements(By.CSS_SELECTOR, 'span.pui__gfuUIT')
        revisit = revisit_elements[1].text.strip() if len(revisit_elements) > 1 else ''
        review_cnt = r.find_element(By.CSS_SELECTOR, 'span.pui__WN-kAf').text.strip()
        url = r.find_element(By.CSS_SELECTOR, 'a[href*="/my"]').get_attribute('href')
        # revisit "번째 방문" 문자 제거
        if revisit:
            revisit = int(revisit[:-5])
        
        # review_cnt "리뷰 " 문자 제거
        if review_cnt:
            review_cnt = int(review_cnt[3:])
            # 태그 저장
        i_tags = [tag.text.strip() for tag in r.find_elements(By.CSS_SELECTOR, 'div.pui__HLNvmI')]
        i_tags = str(i_tags)
        if "+" not in i_tags: # 태그 0개, 1개일 때
            i_tag = [tag.text.strip() for tag in r.find_elements(By.CSS_SELECTOR, 'div.pui__HLNvmI')]
        else: # 태그 2개 이상일 때
            # 태그 더보기 버튼 누르기
            tag_button = r.find_element(By.CSS_SELECTOR, 'a.pui__jhpEyP.pui__ggzZJ8')
            driver.execute_script("arguments[0].click();", tag_button)
            time.sleep(1)
            # 태그 여러 개 잘라서 리스트에 저장
            i_tag = [tag.text.strip() for tag in r.find_elements(By.CSS_SELECTOR, 'div.pui__HLNvmI span.pui__jhpEyP')]
            # 중복 없이 리뷰 데이터 저장
        list_sheet.append([nickname, content, date, revisit, ", ".join(i_tag), review_cnt, url])
        
if 'driver' in locals():
    driver.quit()



        


In [50]:
print(file_name)


naver_review_2025-05-17_20-45-59.xlsx
